In [47]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [48]:

url1 = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018','2019']

#donwloading xg data for premier league players
urlepl = url1 + '/' + leagues[1] + '/' + seasons[-1]
res = requests.get(urlepl)
soup = BeautifulSoup(res.content, "lxml")
scripts = soup.find_all('script')

string_with_json_obj = ''
for el in scripts:
    if 'playersData' in el.text:
        string_with_json_obj = el.text.strip()
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')
data1 = json.loads(json_data)
df = pd.DataFrame(data1)
df = df.set_index('player_name')


In [49]:
#downloading fpl data
from pandas.io.json import json_normalize
link = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(link)
data = json.loads(response.text)
df1 = json_normalize(data['elements'])
df1['full_name']=df1['first_name'] + ' ' + df1['second_name']
df1 = df1.reset_index().set_index('full_name')
        

In [50]:
#combining the two dataframes, need to do some data cleaning and data not pulling through for some players due to names eg Wesley
df2 = pd.concat([df, df1], axis=1, sort=False)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 625 entries, Jamie Vardy to Taylor Perry
Data columns (total 71 columns):
assists                         428 non-null object
games                           428 non-null object
goals                           428 non-null object
id                              428 non-null object
key_passes                      428 non-null object
npg                             428 non-null object
npxG                            428 non-null object
position                        428 non-null object
red_cards                       428 non-null object
shots                           428 non-null object
team_title                      428 non-null object
time                            428 non-null object
xA                              428 non-null object
xG                              428 non-null object
xGBuildup                       428 non-null object
xGChain                         428 non-null object
yellow_cards                    428 non-null obje

In [21]:
#returns highest point scorers from each position
def max_point_players(df):
    max_points = []
    maxplayercost = 0
    for i in range(1,5):
        dfpos = df.loc[df['element_type']==i]
        player = dfpos['total_points'].idxmax()
        maxplayercost=maxplayercost+df.loc[player]['now_cost']
        max_points.append(player)
    return max_points, maxplayercost
    
    


In [45]:
#basic optimiser for fpl team
def optimal_team(budget,df):
    gk = 2
    defender = 5
    mid = 5
    forw = 3
    positions = {1:1,2:4,3:4,4:2}
    opt_team = []
    i=0
    opt_team=opt_team + max_point_players(df)[0]
    budget=budget-max_point_players(df)[1]
    dftemp=df.copy()
    dftemp['ppm']=dftemp['total_points']/dftemp['now_cost']
    dftemp=dftemp.sort_values(['ppm'],ascending=False)
    while len(opt_team)<15:
        player_pos = dftemp.iloc[i]['element_type']
        player_name = dftemp.iloc[i].name
        player_cost = dftemp.iloc[i]['now_cost']
        if player_name not in opt_team:
            if budget>=player_cost:
                if positions[player_pos]>0:
                    opt_team.append(player_name)
                    positions[player_pos]=positions[player_pos]-1
                    budget=budget-player_cost
        i+=1
    return opt_team, budget  
        

In [46]:
df3=df1[['element_type','now_cost','total_points']]
#test functions
max_point_players(df3)
optimal_team(1000,df3)

(['Mathew Ryan',
  'John Lundstram',
  'Sadio Mané',
  'Jamie Vardy',
  'George Baldock',
  'Aaron Ramsdale',
  'Çaglar Söyüncü',
  'Lewis Dunk',
  "Jack O'Connell",
  'Tammy Abraham',
  'Scott McTominay',
  'Teemu Pukki',
  'Youri Tielemans',
  'John McGinn',
  'Mason Mount'],
 65.0)